In [21]:
from IPython import __version__ as ipython_version
from pandas import __version__ as pandas_version
from bokeh import __version__ as bokeh_version

print("IPython - %s" % ipython_version)
print("Pandas - %s" % pandas_version)
print("Bokeh - %s" % bokeh_version)


IPython - 6.1.0
Pandas - 0.22.0
Bokeh - 0.12.14


In [2]:
import pandas as pd
import numpy as np


In [3]:
stgrnk = pd.read_csv("VWSTAGERANKING.csv")

# stgrnk = stgrnk.loc[(stgrnk['status']=='A')]

# stgrnk


In [4]:
stgrnk.drop(
    ["code_table", "MEDIUM_DESC", "Converted/Confirmed/Accepted/Require SepDate"],
    inplace=True,
    axis=1,
)

# stgrnk


In [5]:
stg_hist_dtype = {
    "PEOPLE_CODE_ID": str,
    "ACADEMIC_YEAR": str,
    "ACADEMIC_TERM": str,
    "ACADEMIC_SESSION": str,
    "FIELD_ID": np.int64,
}
date_cols = ["FIELD_DATE", "REVISION_DATE", "REVISION_TIME"]
stg_hist = pd.read_csv(
    "STAGEHISTORY.csv",
    dtype=stg_hist_dtype,
    parse_dates=date_cols,
    usecols=[
        "PEOPLE_CODE_ID",
        "ACADEMIC_YEAR",
        "ACADEMIC_TERM",
        "ACADEMIC_SESSION",
        "FIELD_ID",
        "FIELD_DATE",
        "REVISION_DATE",
        "REVISION_TIME",
    ],
)

print(stg_hist.shape)

# stg_hist.loc[(stg_hist['ACADEMIC_YEAR']=='2018') & (stg_hist['ACADEMIC_TERM']=='FALL')]


(279835, 8)


In [ ]:
stg_hist.dtypes


In [6]:
stg_hist["Year_Term"] = (
    stg_hist["ACADEMIC_YEAR"] + "." + stg_hist["ACADEMIC_TERM"].str.title()
)


In [ ]:
stg_hist.head()


In [ ]:
##stg_hist['year_term'] = stg_hist.apply(lambda r : r['ACADEMIC_YEAR'] + '.' + r['ACADEMIC_TERM'].title())


In [7]:
stg_hist["Revision"] = stg_hist.apply(
    lambda r: pd.datetime.combine(r["REVISION_DATE"].date(), r["REVISION_TIME"].time()),
    1,
)

# stg_hist.loc[(stg_hist['ACADEMIC_YEAR']=='2018') & (stg_hist['ACADEMIC_TERM']=='FALL')]


In [8]:
stg_hist["ACADEMIC_YEAR"] = pd.to_numeric(
    stg_hist["ACADEMIC_YEAR"], errors="coerce", downcast="integer"
)

stg_hist.loc[(stg_hist["ACADEMIC_YEAR"].isnull())].shape


(152, 10)

In [9]:
stg_hist = stg_hist.loc[
    pd.to_numeric(
        stg_hist["ACADEMIC_YEAR"], errors="coerce", downcast="integer"
    ).notnull()
]

stg_hist.shape


(279683, 10)

In [ ]:
stg_hist.dtypes


In [10]:
stg_hist.drop(["FIELD_DATE", "REVISION_DATE", "REVISION_TIME"], inplace=True, axis=1)

# stg_hist.loc[(stg_hist['ACADEMIC_YEAR']==2018) & (stg_hist['ACADEMIC_TERM']=='FALL')]


In [11]:
stg_hist["Week_Number"] = stg_hist["Revision"].dt.week

# stg_hist.loc[(stg_hist['ACADEMIC_YEAR']==2018) & (stg_hist['ACADEMIC_TERM']=='FALL')]


In [ ]:
from datetime import datetime, date

print(date(int("2017"), 9, 1).isocalendar()[1])


In [ ]:
def admissions_week(df):
    week_num = df["Week_Number"]
    academic_year = df["ACADEMIC_YEAR"]
    admissions_week_num = week_num - (date(int(academic_year), 9, 1).isocalendar()[1])
    if admissions_week_num < 0:
        admissions_week_num = 53 + admissions_week_num
    return admissions_week_num


In [ ]:
from datetime import datetime, date

# stg_hist['Admissions_Week'] = stg_hist.apply(admissions_week(stg_hist),1)
stg_hist["Admiss_Week_Begin"] = stg_hist.apply(
    lambda r: (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1]), 1
)
# stg_hist['Admissions_Week_Begin'] = stg_hist.apply(lambda r : (date(int(r['ACADEMIC_YEAR']), 9, 1).isocalendar()[1]),1)

stg_hist.loc[
    (stg_hist["ACADEMIC_YEAR"] == 2018) & (stg_hist["ACADEMIC_TERM"] == "FALL")
]


In [ ]:
from datetime import datetime, date

stg_hist["Admissions_Week"] = stg_hist.apply(
    lambda r: r["Week_Number"] - (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1]),
    1,
)

stg_hist.loc[
    (stg_hist["ACADEMIC_YEAR"] == 2018) & (stg_hist["ACADEMIC_TERM"] == "FALL")
]


In [12]:
from datetime import datetime, date

stg_hist["Admissions_Week"] = stg_hist.apply(
    lambda r: (
        r["Week_Number"] - (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1])
    )
    if (r["Week_Number"] > (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1]))
    else (
        53 + r["Week_Number"] - (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1])
    ),
    axis=1,
)


In [ ]:
stg_hist.loc[
    (stg_hist["ACADEMIC_YEAR"] == 2018) & (stg_hist["ACADEMIC_TERM"] == "FALL")
]


In [ ]:
stg_hist.dtypes


In [13]:
stage_data = pd.merge(
    stg_hist, stgrnk, left_on=["FIELD_ID"], right_on=["STAGERANKING_ID"], how="left"
)

print(stage_data.shape)

# stage_data.loc[(stage_data['ACADEMIC_YEAR']==2018) & (stage_data['ACADEMIC_TERM']=='FALL')]


(279683, 16)


In [ ]:
%cd '/home/jw/Documents/PaulSmiths/IR/Admissions'
%run -i 'admissions.py'

In [ ]:
stage_data.loc[
    (stage_data["ACADEMIC_YEAR"] == 2018) & (stage_data["ACADEMIC_TERM"] == "FALL")
]


In [ ]:
stage_data.dtypes


In [ ]:
print(stage_data.columns)


In [ ]:
print("field_value", stage_data["field_value"].unique())

print("status", stage_data["status"].unique())


In [ ]:
sd1 = stage_data.loc[
    (
        stage_data["field_value"].isin(
            [
                "300",
                "ACC",
                "ACXL",
                "CANC",
                "DEF",
                "DEFR",
                "DENY",
                "DPAC",
                "TRDP",
                "TRPD",
                "TRNS",
                "WAIT",
            ]
        )
    )
]
sd1 = sd1[["PEOPLE_CODE_ID", "Year_Term", "Admissions_Week", "field_value", "status"]]

print(sd1.shape)
print("Canceled", stage_data["Canceled"].unique())

sd1.head()


In [ ]:
print("Canceled", stage_data["Canceled"].unique())

sd1.head()


In [18]:
ad_keep_values = [
    "300",
    "ACC",
    "ACXL",
    "CANC",
    "DEF",
    "DEFR",
    "DENY",
    "DPAC",
    "TRDP",
    "TRPD",
    "TRNS",
    "WAIT",
]
ad_keep_cols = [
    "PEOPLE_CODE_ID",
    "Year_Term",
    "Admissions_Week",
    "field_value",
    "status",
]
sd1 = stage_data.loc[(stage_data["field_value"].isin(ad_keep_values))]
sd1 = sd1[ad_keep_cols]

print(sd1.shape)
sd1.head()


(37676, 5)


,PEOPLE_CODE_ID,Year_Term,Admissions_Week,field_value,status
2,P000000006,2000.Spring,8,ACC,A
16,P000000006,2000.Spring,8,ACC,A
34,P000000007,2003.Spring,7,ACC,A
36,P000000007,2003.Spring,7,ACC,A
40,P000000013,2004.Fall,7,ACC,A


In [19]:
# sd1[(sd1['field_value']=='500')] = 'Deposited'
# sd1[(sd1['field_value']=='ACC')] = 'Accepted'
# get this table from PC
admission_status = {
    "300": "Applied",
    "ACC": "Accepted",
    "ACXL": "Canceled",
    "CANC": "Canceled",
    "DEF": "Canceled",
    "DEFR": "Canceled",
    "DENY": "Canceled",
    "DPAC": "Deposited",
    "TRDP": "Deposited",
    "TRPD": "Deposited",
    "TRNS": "Accepted",
    "WAIT": "Accepted",
}
# print(list(admission_status.items()))
adm_stat = pd.DataFrame(
    list(admission_status.items()), columns=["field_value", "admission_status"]
)
# adm_stat = pd.Series(admission_status, name='admission_status')
# adm_stat.index.name = 'field_value'
# adm_stat.reset_index()
print(adm_stat)


   field_value admission_status
0          300          Applied
1          ACC         Accepted
2         ACXL         Canceled
3         CANC         Canceled
4          DEF         Canceled
5         DEFR         Canceled
6         DENY         Canceled
7         DPAC        Deposited
8         TRDP        Deposited
9         TRPD        Deposited
10        TRNS         Accepted
11        WAIT         Accepted


In [ ]:
sd1 = pd.merge(sd1, adm_stat, on=["field_value"], how="left")

print(sd1.shape)
sd1.head()


In [ ]:
print("status", sd1["status"].unique())
sd1.drop(["field_value", "status"], inplace=True, axis=1)
# or sd1 = sd1[['Year_Term', 'PEOPLE_CODE_ID', 'Admissions_Week', 'admission_status']]

sd1 = sd1.drop_duplicates(
    ["PEOPLE_CODE_ID", "Year_Term", "Admissions_Week", "admission_status"]
)

print(sd1.shape)

sd1.head()


In [ ]:
print("admission_status", sd1["admission_status"].unique())


In [ ]:
sd_test = sd1.iloc[:10]
print(sd_test)


In [ ]:
sdt1 = (
    sd1.reset_index()
    .set_index(["Year_Term", "PEOPLE_CODE_ID", "admission_status"])
    .drop(["index"], axis=1)
)
sdt1


In [20]:
sd1 = (
    pd.merge(sd1, adm_stat, on=["field_value"], how="left")
    .drop(["field_value", "status"], axis=1)
    .drop_duplicates(
        ["PEOPLE_CODE_ID", "Year_Term", "Admissions_Week", "admission_status"]
    )
    .reset_index()
    .set_index(["Year_Term", "PEOPLE_CODE_ID", "admission_status"])
    .drop(["index"], axis=1)
)
print(sd1.shape)
sd1.head()


(28424, 1)


,,,Admissions_Week
Year_Term,PEOPLE_CODE_ID,admission_status,
2000.Spring,P000000006,Accepted,8
2003.Spring,P000000007,Accepted,7
2004.Fall,P000000013,Accepted,7
2000.Spring,P000000016,Accepted,8
2001.Spring,P000000017,Accepted,8


In [ ]:
sdt1.index


In [21]:
# sd2 = sd1.loc[(sd1['Year_Term'].isin(['2010.Fall', '2011.Fall', '2012.Fall', '2013.Fall', '2014.Fall', '2015.Fall', '2016.Fall', '2017.Fall']))]
sd2 = sd1.loc[(sd1["Year_Term"].isin(["2014.Fall", "2015.Fall"]))]

sd3 = sd2.groupby(["Year_Term", "Admissions_Week", "admission_status"]).count()
print(sd3.shape)
print(sd3.head())
print(sd3.iloc[0])


(390, 1)
                                            PEOPLE_CODE_ID
Year_Term Admissions_Week admission_status                
2014.Fall 1               Accepted                       1
                          Applied                       10
                          Canceled                       8
          2               Accepted                       7
                          Applied                        8
PEOPLE_CODE_ID    1
Name: (2014.Fall, 1, Accepted), dtype: int64


In [22]:
sd4 = sd2.sort_values(["Year_Term", "PEOPLE_CODE_ID", "Admissions_Week"])
# sd4.head(15)

print(sd4.shape)
sd4


(8809, 4)


,PEOPLE_CODE_ID,Year_Term,Admissions_Week,admission_status
4503,P000010351,2014.Fall,37,Applied
4504,P000010351,2014.Fall,47,Canceled
5830,P000013143,2014.Fall,46,Applied
5831,P000013143,2014.Fall,48,Accepted
5832,P000013143,2014.Fall,49,Deposited
5833,P000013143,2014.Fall,49,Accepted
5834,P000013143,2014.Fall,53,Deposited
5835,P000013143,2014.Fall,53,Canceled
9855,P000021911,2014.Fall,22,Applied
9856,P000021911,2014.Fall,53,Canceled


In [23]:
sd5 = sd4.drop_duplicates(
    ["Year_Term", "PEOPLE_CODE_ID", "admission_status"], keep="first"
)

print(sd5.shape)
sd5.head(15)


(8630, 4)


,PEOPLE_CODE_ID,Year_Term,Admissions_Week,admission_status
4503,P000010351,2014.Fall,37,Applied
4504,P000010351,2014.Fall,47,Canceled
5830,P000013143,2014.Fall,46,Applied
5831,P000013143,2014.Fall,48,Accepted
5832,P000013143,2014.Fall,49,Deposited
5835,P000013143,2014.Fall,53,Canceled
9855,P000021911,2014.Fall,22,Applied
9856,P000021911,2014.Fall,53,Canceled
15403,P000026232,2014.Fall,21,Applied
15404,P000026232,2014.Fall,22,Accepted


In [30]:
print(sdt1.unstack())


ValueError: Index contains duplicate entries, cannot reshape

In [36]:
sdt2 = (
    sd1.reset_index().set_index(["Year_Term", "PEOPLE_CODE_ID"]).drop(["index"], axis=1)
)
sdt2


Admissions_Week admission_status
Year_Term   PEOPLE_CODE_ID                                  
2000.Spring P000000006                    8         Accepted
2003.Spring P000000007                    7         Accepted
2004.Fall   P000000013                    7         Accepted
2000.Spring P000000016                    8         Accepted
2001.Spring P000000017                    8         Accepted
2000.Spring P000000020                    8         Accepted
            P000000023                    8         Accepted
2001.Summer P000000024                    8         Accepted
2003.Spring P000000024                    7         Accepted
2008.Fall   P000000030                   33         Accepted
            P000000030                   33          Applied
            P000000030                   34          Applied
            P000000030                   34         Accepted
2002.Spring P000000031                    8         Accepted
2001.Spring P000000034                    8         Accepted
2003.Spring P000000035                    7         Accepted
2004.Fall   P000000038                    7         Accepted
2004.Spring P000000049                    7         Accepted
2005.Spring P000000049                    8         Accepted
2000.Fall   P000000055                    8         Accepted
            P000000083                    8         Accepted
2002.Summer P000000083                    8         Accepted
2000.Spring P000000091                    8         Accepted
2001.Spring P000000091                    8         Accepted
2000.Spring P000000093                    8         Accepted
2001.Spring P000000093                    8         Accepted
2000.Spring P000000094                    8         Accepted
2002.Summer P000000094                    8         Accepted
2000.Spring P000000317                    8         Accepted
2001.Fall   P000000332                    8         Accepted
...                                     ...              ...
2018.Fall   P000054624                   11         Accepted
            P000054625                   11          Applied
            P000054625                   11         Accepted
            P000054626                   11         Accepted
            P000054627                   11          Applied
            P000054627                   11         Accepted
            P000054628                   11         Accepted
            P000054629                   11         Accepted
            P000054630                   11          Applied
            P000054630                   11         Accepted
            P000054631                   11         Accepted
            P000054632                   11         Accepted
            P000054633                   11         Accepted
            P000054634                   11         Accepted
            P000054635                   11         Accepted
            P000054636                   11         Accepted
            P000054641                   11          Applied
            P000054641                   11         Accepted
            P000054642                   11          Applied
            P000054642                   11         Accepted
            P000054643                   11         Accepted
            P000054644                   11         Accepted
            P000054645                   11         Accepted
            P000054646                   11         Accepted
            P000054647                   11         Accepted
            P000054648                   11         Accepted
            P000054649                   11         Accepted
            P000054650                   11         Accepted
            P000054651                   11         Accepted
            P000054652                   11         Accepted

[28424 rows x 2 columns]

In [56]:
sd6 = (
    sd1.loc[(sd1["Year_Term"].isin(["2014.Fall", "2015.Fall"]))]
    .sort_values(["Year_Term", "PEOPLE_CODE_ID", "Admissions_Week"])
    .drop_duplicates(["Year_Term", "PEOPLE_CODE_ID", "admission_status"], keep="first")
    .reset_index()
    .set_index(["Year_Term", "PEOPLE_CODE_ID", "admission_status"])
    .drop(["index"], axis=1)
)
print(sd6.head())
print(sd6.unstack(level=-1).head())
# sd6.pivot(values='Admissions_Week', index=['Year_Term', 'PEOPLE_CODE_ID'], columns='admission_status' )


                                           Admissions_Week
Year_Term PEOPLE_CODE_ID admission_status                 
2014.Fall P000010351     Applied                        37
                         Canceled                       47
          P000013143     Applied                        46
                         Accepted                       48
                         Deposited                      49
                         Admissions_Week                           
admission_status                Accepted Applied Canceled Deposited
Year_Term PEOPLE_CODE_ID                                           
2014.Fall P000010351                 NaN    37.0     47.0       NaN
          P000013143                48.0    46.0     53.0      49.0
          P000021911                 NaN    22.0     53.0       NaN
          P000026232                22.0    21.0      NaN      36.0
          P000026411                30.0    20.0      NaN      39.0


<b>ACADEMIC table</b>

In [ ]:
%cd '/home/jw/Documents/PaulSmiths/IR/PowerCampus'
%pwd

In [25]:
academic = pd.read_csv("ACADEMIC.csv")

academic.head()


C:\Anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,10,12,54,58,61,68,73,74,77,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PEOPLE_CODE,PEOPLE_ID,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,PROGRAM,DEGREE,CURRICULUM,COLLEGE,...,PROGRAM_END_DATE,MET_ENGLISH_REQUIREMENT,DROP_BELOW_FULL_CODE,AUTH_START_DATE,AUTH_END_DATE,PROTECT_COUNSELOR,ENROLLMENT_STATUS_DATE,INACTIVE_STATUS_DATE,FULL_PART_NON_WITHDRAWN,FULL_PART_DISP_WITHDRAWN
0,P,6,P000000006,2000,SPRING,NaN,U,AAS,UNDM,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LESS,1
1,P,6,P000000006,2000,SPRING,MAIN,U,AAS,UNDM,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LESS,1
2,P,6,P000000006,2011,SUMMER,NaN,U,AAS,UNDM,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LESS,1
3,P,6,P000000006,2011,SUMMER,CULN,U,AAS,UNDM,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LESS,1
4,P,7,P000000007,2003,SPRING,NaN,U,AAS,UNDM,NaN,...,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,LESS,1


In [26]:
academic.columns


Index(['PEOPLE_CODE', 'PEOPLE_ID', 'PEOPLE_CODE_ID', 'ACADEMIC_YEAR',
       'ACADEMIC_TERM', 'ACADEMIC_SESSION', 'PROGRAM', 'DEGREE', 'CURRICULUM',
       'COLLEGE', 'DEPARTMENT', 'CLASS_LEVEL', 'NONTRAD_PROGRAM', 'POPULATION',
       'ADVISOR', 'ADMIT_YEAR', 'ADMIT_TERM', 'ADMIT_SESSION', 'ADMIT_DATE',
       'MATRIC', 'MATRIC_YEAR', 'MATRIC_TERM', 'MATRIC_SESSION', 'MATRIC_DATE',
       'FULL_PART', 'ACADEMIC_STANDING', 'REGISTER_LIMIT', 'EXPECT_GRAD_MM',
       'EXPECT_GRAD_YYYY', 'ENROLL_SEPARATION', 'SEPARATION_DATE', 'CREDITS',
       'CREATE_DATE', 'CREATE_TIME', 'CREATE_OPID', 'CREATE_TERMINAL',
       'REVISION_DATE', 'REVISION_TIME', 'REVISION_OPID', 'REVISION_TERMINAL',
       'ABT_JOIN', 'PREREG_VALIDATE', 'PREREG_VAL_WHO', 'PREREG_VAL_DATE',
       'REG_VALIDATE', 'REG_VAL_WHO', 'REG_VAL_DATE', 'GRADUATED',
       'GRADUATED_YEAR', 'GRADUATED_TERM', 'GRADUATED_SESSION', 'ORG_CODE_ID',
       'ACADEMIC_FLAG', 'APPLICATION_FLAG', 'APP_STATUS', 'APP_STATUS_DATE',
       'APP

In [27]:
academic_dtype = {
    "PEOPLE_CODE_ID": str,
    "ACADEMIC_YEAR": str,
    "ACADEMIC_TERM": str,
    "ACADEMIC_SESSION": str,
    "APPLICATION_FLAG": str,
    "APP_STATUS": str,
}
date_cols = [
    "APPLICATION_DATE",
    "APP_STATUS_DATE",
    "APP_DECISION_DATE",
    "REVISION_DATE",
    "REVISION_TIME",
]
academic = pd.read_csv(
    "ACADEMIC.csv",
    dtype=academic_dtype,
    parse_dates=date_cols,
    usecols=[
        "PEOPLE_CODE_ID",
        "ACADEMIC_YEAR",
        "ACADEMIC_TERM",
        "ACADEMIC_SESSION",
        "POPULATION",
        "INQUIRY_FLAG",
        "APPLICATION_FLAG",
        "APPLICATION_DATE",
        "APP_STATUS",
        "APP_STATUS_DATE",
        "APP_DECISION",
        "APP_DECISION_DATE",
        "REVISION_DATE",
        "REVISION_TIME",
    ],
)

print(academic.shape)


(122300, 14)


In [28]:
from datetime import date

academic["Year_Term"] = (
    academic["ACADEMIC_YEAR"] + "." + academic["ACADEMIC_TERM"].str.title()
)
print(academic.shape)

academic["ACADEMIC_YEAR"] = pd.to_numeric(
    academic["ACADEMIC_YEAR"], errors="coerce", downcast="integer"
)
academic = academic.loc[
    pd.to_numeric(
        academic["ACADEMIC_YEAR"], errors="coerce", downcast="integer"
    ).notnull()
]
print(academic.shape)

academic["Revision"] = academic.apply(
    lambda r: pd.datetime.combine(r["REVISION_DATE"].date(), r["REVISION_TIME"].time()),
    1,
)
academic.drop(["REVISION_DATE", "REVISION_TIME"], inplace=True, axis=1)
academic["Week_Number"] = academic["Revision"].dt.week
academic["Admissions_Week"] = academic.apply(
    lambda r: (
        r["Week_Number"] - (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1])
    )
    if (r["Week_Number"] > (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1]))
    else (
        53 + r["Week_Number"] - (date(int(r["ACADEMIC_YEAR"]), 9, 1).isocalendar()[1])
    ),
    axis=1,
)
print(academic.shape)


(122300, 15)
(122155, 15)
(122155, 16)


In [29]:
academic = academic.loc[
    ((academic["ACADEMIC_TERM"] == "FALL") | (academic["ACADEMIC_TERM"] == "SPRING"))
    & (academic["ACADEMIC_SESSION"] == "MAIN")
]
print(academic.shape)


(68679, 16)


In [31]:
print(academic.loc[(academic["ACADEMIC_TERM"].isin(["FALL", "SPRING"]))].shape)
academic.loc[(academic["ACADEMIC_TERM"].isin(["FALL", "SPRING"]))].head()


(68679, 16)


,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,APPLICATION_FLAG,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE,INQUIRY_FLAG,APPLICATION_DATE,Year_Term,Revision,Week_Number,Admissions_Week
1,P000000006,2000.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2000.Spring,2007-06-19 11:57:40,25,43
5,P000000007,2003.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2003.Spring,2007-05-15 17:20:07,20,37
7,P000000013,2004.0,FALL,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2004.Fall,2008-09-16 10:33:00,38,2
11,P000000016,2000.0,FALL,MAIN,NaN,N,500,NaT,ACC,NaT,N,NaT,2000.Fall,2007-06-25 15:58:31,26,44
13,P000000016,2000.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2000.Spring,2007-06-25 15:58:31,26,44


In [32]:
print(academic.shape)
academic.head()


(68679, 16)


,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,APPLICATION_FLAG,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE,INQUIRY_FLAG,APPLICATION_DATE,Year_Term,Revision,Week_Number,Admissions_Week
1,P000000006,2000.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2000.Spring,2007-06-19 11:57:40,25,43
5,P000000007,2003.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2003.Spring,2007-05-15 17:20:07,20,37
7,P000000013,2004.0,FALL,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2004.Fall,2008-09-16 10:33:00,38,2
11,P000000016,2000.0,FALL,MAIN,NaN,N,500,NaT,ACC,NaT,N,NaT,2000.Fall,2007-06-25 15:58:31,26,44
13,P000000016,2000.0,SPRING,MAIN,NaN,Y,500,NaT,ACC,NaT,Y,NaT,2000.Spring,2007-06-25 15:58:31,26,44


In [33]:
app_data = academic.loc[
    ~(academic["POPULATION"].isin(["AVDSTU", "NOND"]))
    & ((academic["INQUIRY_FLAG"] == "Y") | (academic["APPLICATION_FLAG"] == "Y"))
]
print(app_data.shape)

academic.loc[(academic["Year_Term"] == "2018.Fall")]


(32890, 16)


,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,APPLICATION_FLAG,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE,INQUIRY_FLAG,APPLICATION_DATE,Year_Term,Revision,Week_Number,Admissions_Week
75941,P000026933,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-10 15:22:18.407,24,42
75972,P000026935,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-10 15:22:05.000,24,42
75981,P000026937,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-10 15:22:33.577,24,42
80397,P000029261,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-10 15:13:03.327,24,42
81814,P000029731,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-05 16:09:00.217,23,41
81818,P000029736,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-06-05 16:05:44.420,23,41
91197,P000036259,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-04-26 08:10:55.593,17,35
91201,P000036263,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-04-26 08:19:07.677,17,35
91202,P000036264,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-04-26 08:23:05.860,17,35
91203,P000036265,2018.0,FALL,MAIN,NaN,N,NaN,NaT,NaN,NaT,Y,NaT,2018.Fall,2013-04-26 08:24:32.737,17,35


In [34]:
applied = app_data[app_data["APP_STATUS"].notnull()]

print(applied.shape)
applied.loc[(applied["Year_Term"] == "2018.Fall")]


(14297, 16)


,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,APPLICATION_FLAG,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE,INQUIRY_FLAG,APPLICATION_DATE,Year_Term,Revision,Week_Number,Admissions_Week
105617,P000045209,2018.0,FALL,MAIN,NaN,Y,COMP,2017-10-11,ACC,2017-10-11,N,2017-08-31,2018.Fall,2017-11-17 14:39:41.597,46,11
108461,P000047242,2018.0,FALL,MAIN,NaN,Y,300,2017-10-30,ACC,2017-11-17,N,2017-11-17,2018.Fall,2017-11-17 14:39:41.203,46,11
120175,P000053326,2018.0,FALL,MAIN,NaN,Y,500,2016-10-30,DPAC,2016-11-30,N,2017-07-17,2018.Fall,2017-07-17 08:21:27.417,29,47
120783,P000053572,2018.0,FALL,MAIN,NaN,Y,300,2017-08-01,ACC,2017-10-13,N,2017-11-17,2018.Fall,2017-11-17 10:47:56.077,46,11
120795,P000053576,2018.0,FALL,MAIN,NaN,Y,COMP,2017-10-27,ACC,2017-10-27,N,2017-10-24,2018.Fall,2017-11-17 14:05:23.907,46,11
121248,P000053870,2018.0,FALL,MAIN,NaN,Y,300,2017-02-06,ACC,2017-02-07,N,2017-02-06,2018.Fall,2017-06-22 12:43:54.497,25,43
121347,P000053942,2018.0,FALL,MAIN,NaN,Y,COMP,2017-11-03,ACC,2017-11-03,N,2017-10-30,2018.Fall,2017-11-17 14:01:28.373,46,11
121757,P000054180,2018.0,FALL,MAIN,NaN,Y,500,2017-09-01,DPAC,2017-09-01,N,2017-09-01,2018.Fall,2017-09-01 09:17:49.430,35,53
121951,P000054294,2018.0,FALL,MAIN,NaN,Y,COMP,2017-08-21,ACC,2017-08-21,N,2017-08-21,2018.Fall,2017-08-21 07:57:23.963,34,52
122119,P000054428,2018.0,FALL,MAIN,NaN,Y,500,2017-08-31,DPAC,2017-08-30,N,2017-08-30,2018.Fall,2017-08-31 08:27:27.327,35,53


In [35]:
accepted = app_data[app_data["APP_DECISION"].notnull()]

print(accepted.shape)
accepted.loc[(accepted["Year_Term"] == "2018.Fall")]


(14201, 16)


,PEOPLE_CODE_ID,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,POPULATION,APPLICATION_FLAG,APP_STATUS,APP_STATUS_DATE,APP_DECISION,APP_DECISION_DATE,INQUIRY_FLAG,APPLICATION_DATE,Year_Term,Revision,Week_Number,Admissions_Week
105617,P000045209,2018.0,FALL,MAIN,NaN,Y,COMP,2017-10-11,ACC,2017-10-11,N,2017-08-31,2018.Fall,2017-11-17 14:39:41.597,46,11
108461,P000047242,2018.0,FALL,MAIN,NaN,Y,300,2017-10-30,ACC,2017-11-17,N,2017-11-17,2018.Fall,2017-11-17 14:39:41.203,46,11
120175,P000053326,2018.0,FALL,MAIN,NaN,Y,500,2016-10-30,DPAC,2016-11-30,N,2017-07-17,2018.Fall,2017-07-17 08:21:27.417,29,47
120783,P000053572,2018.0,FALL,MAIN,NaN,Y,300,2017-08-01,ACC,2017-10-13,N,2017-11-17,2018.Fall,2017-11-17 10:47:56.077,46,11
120795,P000053576,2018.0,FALL,MAIN,NaN,Y,COMP,2017-10-27,ACC,2017-10-27,N,2017-10-24,2018.Fall,2017-11-17 14:05:23.907,46,11
121248,P000053870,2018.0,FALL,MAIN,NaN,Y,300,2017-02-06,ACC,2017-02-07,N,2017-02-06,2018.Fall,2017-06-22 12:43:54.497,25,43
121347,P000053942,2018.0,FALL,MAIN,NaN,Y,COMP,2017-11-03,ACC,2017-11-03,N,2017-10-30,2018.Fall,2017-11-17 14:01:28.373,46,11
121757,P000054180,2018.0,FALL,MAIN,NaN,Y,500,2017-09-01,DPAC,2017-09-01,N,2017-09-01,2018.Fall,2017-09-01 09:17:49.430,35,53
121951,P000054294,2018.0,FALL,MAIN,NaN,Y,COMP,2017-08-21,ACC,2017-08-21,N,2017-08-21,2018.Fall,2017-08-21 07:57:23.963,34,52
122119,P000054428,2018.0,FALL,MAIN,NaN,Y,500,2017-08-31,DPAC,2017-08-30,N,2017-08-30,2018.Fall,2017-08-31 08:27:27.327,35,53


In [36]:
print("APP_STATUS", applied["APP_STATUS"].unique())
print("APP_DECISION", accepted["APP_DECISION"].unique())


APP_STATUS ['500' '300' 'ACXL' 'CANC' 'DEFR' 'COMP' 'nosh']
APP_DECISION ['ACC' 'DPAC' 'PEND' 'CANC' 'DEF' 'nosh' 'TRDP' 'TRNS' 'DENY']
